In [20]:
#安裝套件
!pip install PyMySQL
!pip install boto3

In [21]:
#引用套件
import boto3
from botocore.exceptions import ClientError
import json
import os
from datetime import datetime 

import calendar 
import decimal
os.environ["TZ"] = "UTC" 

d = datetime.utcnow()

# 先設定endpoint_url
epurl="http://cc104.dynamodb.local:8000"

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

# 用boto3的session方法調用container中.aws/目錄下的環境變數設定檔，並存入變數user_session中
user_session=boto3.session.Session(profile_name='user1')
print("="*40)
# boto3透過container的3個aws環境變數，來連線dynamodb
dynamodb = user_session.resource('dynamodb', endpoint_url=epurl)
print("UTC time : ",d,"~~連線成功~~")
print("="*40)

UTC time :  2019-02-10 07:00:24.517119 ~~連線成功~~


In [22]:
# 創建資料表
try:
    table = dynamodb.create_table(
        #設定資料表名稱
        TableName='Movies3',
        #設定Partition key和sort key
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'
            }

        ],
        #Partition key和sort key屬性
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            }
        ],
        #設定讀寫能力單元(RCU / WCU 這兩個值會影響效能，也會依據需求收費)
        ProvisionedThroughput={
            #設定每次讀取單位
            'ReadCapacityUnits': 2,
            #設定每次寫入單位
            'WriteCapacityUnits': 2
        }
    )
    print("="*40)
    table = dynamodb.Table('Movies3')
    print(table,"資料表:", table.table_status)
    print("="*40)
except Exception as err:
    print("="*40)
    table = dynamodb.Table('Movies3')
    print(table,"資料表:", table.table_status)
    print("="*40)

dynamodb.Table(name='Movies3') 資料表: ACTIVE


In [23]:
# 載入JSON資料，新增到資料表中
print("將json資料導入movie3資料表中:")
print("="*40)
with open("movies.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        year = int(movie['year'])
        title = movie['title']
        info = movie['info']
        table.put_item(
           Item={
               'year': year,
               'title': title,
               'info': json.dumps(str(info))
            }
        )
        print(year,title,"....新增成功")
print("="*40)

將json資料導入movie3資料表中:
2013 Rush ....新增成功
2013 Thor: The Dark World ....新增成功
2013 World War Z ....新增成功


In [24]:
# 用put_item方法增加資料到資料表中
table = dynamodb.Table('Movies3')
title = "Transformers: Age of Extinction"
year = 2014
response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"A mechanic and his daughter make a discovery that brings down Autobots and Decepticons - and a paranoid government official - on them.",
            'rating': decimal.Decimal(0)
        }
    }
)
print("="*40)
print("Item：",json.dumps(response, indent=4, cls=DecimalEncoder),"....新增成功")
print("="*40)

Item： {
    "ResponseMetadata": {
        "RequestId": "c2ee395e-1db0-4f03-ab9f-a3b352d07d79",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "c2ee395e-1db0-4f03-ab9f-a3b352d07d79",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
} ....新增成功


In [25]:
# 用get_item方法搜尋資料表中的資料
title = "Rush"
year = 2013
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("查詢",year,title,"結果如下:")
    print("="*40)
    print(json.dumps(item, indent=4, cls=DecimalEncoder))
    print("="*40)

查詢 2013 Rush 結果如下:
{
    "title": "Rush",
    "year": 2013,
    "info": "\"{'directors': ['Ron Howard'], 'release_date': '2013-09-02T00:00:00Z', 'rating': Decimal('8.3'), 'genres': ['Action', 'Biography', 'Drama', 'Sport'], 'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTQyMDE0MTY0OV5BMl5BanBnXkFtZTcwMjI2OTI0OQ@@._V1_SX400_.jpg', 'plot': 'A re-creation of the merciless 1970s rivalry between Formula One rivals James Hunt and Niki Lauda.', 'rank': 2, 'running_time_secs': 7380, 'actors': ['Daniel Bruhl', 'Chris Hemsworth', 'Olivia Wilde']}\""
}


In [26]:
# 用get_item方法搜尋資料表中的資料
title = "Transformers: Age of Extinction"
year = 2014
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("查詢",year,title,"結果如下:")
    print("="*40)
    print(json.dumps(item, indent=4, cls=DecimalEncoder))
    print("="*40)

查詢 2014 Transformers: Age of Extinction 結果如下:
{
    "title": "Transformers: Age of Extinction",
    "year": 2014,
    "info": {
        "rating": 0,
        "plot": "A mechanic and his daughter make a discovery that brings down Autobots and Decepticons - and a paranoid government official - on them."
    }
}


In [27]:
# 用delete_item方法刪除資料表中的資料
title = "Thor: The Dark World"
year = 2013
print("資料",year,title,"刪除中...")
print("="*40)
try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

print("="*40)

資料 2013 Thor: The Dark World 刪除中...
{
    "ResponseMetadata": {
        "RequestId": "bce91a9e-880b-4da1-b097-728c35be601c",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "bce91a9e-880b-4da1-b097-728c35be601c",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


In [28]:
# 用get_item方法搜尋資料表中的資料
title = "Thor: The Dark World"
year = 2013
print("="*40)
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    try:
        item = response['Item']
        print("查詢",year,title,"結果如下:\n")
        print(json.dumps(item, indent=4, cls=DecimalEncoder))
    except KeyError:
        print(year,title,"查無資料....")
print("="*40)

2013 Thor: The Dark World 查無資料....
